In [3]:
"""Imports go here"""

from transformers import ViltProcessor, ViltForQuestionAnswering
import torch
from PIL import Image
import requests

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
#from word2number import w2n


In [4]:
""" Load CSV and split the data """

# Load the CSV file
csv_path = "/kaggle/input/qna-final/qna_final.csv"
df = pd.read_csv(csv_path)

# Get unique Item_IDs
unique_ids = df["Item_ID"].unique()

# Set random seed for reproducibility
random_seed = 42
np.random.seed(random_seed)

# Shuffle and split the unique IDs
train_ids, temp_ids = train_test_split(unique_ids, test_size=0.3, random_state=random_seed)  # 70% train
val_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=random_seed)      # 15% val, 15% test

# Create train, val, and test DataFrames
train_df = df[df["Item_ID"].isin(train_ids)]
val_df = df[df["Item_ID"].isin(val_ids)]
test_df = df[df["Item_ID"].isin(test_ids)]

print(f"Train size: {len(train_df)}, Val size: {len(val_df)}, Test size: {len(test_df)}")


Train size: 16666, Val size: 3663, Test size: 3558


In [5]:
"""Normalizing and adding non-existing answers to label2id (with test leakage prevention)"""
# Load the model and processor
model_config_source = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

# Text-to-number mapping (same as before)
def get_text_to_num_mapping():
    text_to_num = {
        "zero": "0", "one": "1", "two": "2", "three": "3", "four": "4",
        "five": "5", "six": "6", "seven": "7", "eight": "8", "nine": "9",
        "ten": "10", "eleven": "11", "twelve": "12", "thirteen": "13",
        "fourteen": "14", "fifteen": "15", "sixteen": "16", "seventeen": "17",
        "eighteen": "18", "nineteen": "19", "twenty": "20",
    }
    for i in range(21, 1001):
        text_to_num[str(i)] = str(i)
    return text_to_num

text_to_num_map = get_text_to_num_mapping()

def normalize_answer(answer_str):
    normalized = str(answer_str).strip().lower()
    return text_to_num_map.get(normalized, normalized)

# Create DataFrames with .copy()
train_df = df[df["Item_ID"].isin(train_ids)].copy()
val_df = df[df["Item_ID"].isin(val_ids)].copy()
test_df = df[df["Item_ID"].isin(test_ids)].copy()

# Add normalized answers
train_df['normalized_answer'] = train_df['Answer'].apply(normalize_answer)
val_df['normalized_answer'] = val_df['Answer'].apply(normalize_answer)
test_df['normalized_answer'] = test_df['Answer'].apply(normalize_answer)

# --- KEY CHANGE: Only use TRAINING answers to extend label2id ---
all_train_answers = set(train_df['normalized_answer'].unique())
original_label2id = model_config_source.config.label2id

# Extend label2id with NEW TRAINING ANSWERS only
new_answers = [ans for ans in all_train_answers if ans not in original_label2id]
current_max_id = max(original_label2id.values())

extended_label2id = {**original_label2id}
extended_id2label = {**model_config_source.config.id2label}

for idx, ans in enumerate(new_answers, start=current_max_id + 1):
    extended_label2id[ans] = idx
    extended_id2label[idx] = ans

# --- Filter validation/test sets to keep only answers in extended_label2id ---
def filter_to_valid_labels(df, label2id):
    """Remove rows with answers not in label2id"""
    valid_mask = df['normalized_answer'].isin(label2id)
    invalid_count = len(df) - valid_mask.sum()
    if invalid_count > 0:
        print(f"Filtered {invalid_count} rows with unseen answers")
    return df[valid_mask].copy()

val_df_filtered = filter_to_valid_labels(val_df, extended_label2id)
test_df_filtered = filter_to_valid_labels(test_df, extended_label2id)

# --- Final Check ---
print("\nFinal sizes:")
print(f"Train: {len(train_df)}")
print(f"Val (filtered): {len(val_df_filtered)}")
print(f"Test (filtered): {len(test_df_filtered)}")
print(f"Total labels: {len(extended_label2id)}")

Filtered 161 rows with unseen answers
Filtered 236 rows with unseen answers

Final sizes:
Train: 16666
Val (filtered): 3502
Test (filtered): 3322
Total labels: 3464


In [6]:
"""Create a custom dataset class"""

#text_form = w2n.to_words(i)  # Convert integer to text form

class QnADataset(Dataset):
    def __init__(self, dataframe, image_dir, processor, label2id): # Processor is not strictly needed here anymore, but label2id is
        self.dataframe = dataframe
        self.image_dir = image_dir
        # self.processor = processor # Not used directly in __getitem__ anymore
        self.label2id = label2id
        self.text_to_num = self.generate_text_to_num_mapping()

    def generate_text_to_num_mapping(self):
        # (Your existing generate_text_to_num_mapping method - keep as is)
        text_to_num = {
            "zero": "0", "one": "1", "two": "2", "three": "3", "four": "4",
            "five": "5", "six": "6", "seven": "7", "eight": "8", "nine": "9",
            "ten": "10", "eleven": "11", "twelve": "12", "thirteen": "13",
            "fourteen": "14", "fifteen": "15", "sixteen": "16", "seventeen": "17",
            "eighteen": "18", "nineteen": "19", "twenty": "20",
        }
        for i in range(21, 1001):
            text_to_num[str(i)] = str(i)
        return text_to_num

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_path = f"{self.image_dir}/{row['Image_Path']}"
        question_text = row["Question"]  # Keep as raw text
        answer_str = row["Answer"].strip().lower()

        # Convert text-based numbers to numerical strings if needed
        if answer_str in self.text_to_num:
            processed_answer_str = self.text_to_num[answer_str]
        else:
            processed_answer_str = answer_str

        # Load PIL image
        try:
            pil_image = Image.open(image_path).convert("RGB")
        except FileNotFoundError:
            print(f"Error: Image not found at {image_path}")
            # Handle appropriately: skip, return None, or use a placeholder
            # For now, let's re-raise to make it obvious during debugging
            raise
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            raise


        # Encode the answer string to an ID
        if processed_answer_str in self.label2id:
            answer_id = self.label2id[processed_answer_str]
        else:
            # This can be a common issue. Ensure your label2id covers all possible answers
            # or you have a strategy for unknown answers.
            print(f"Warning: Answer '{processed_answer_str}' (original: '{row['Answer']}') not found in label2id mapping. Item index: {idx}, Image: {row['Image_Path']}")
            # Option 1: Raise error (as you had)
            # raise ValueError(f"Answer '{processed_answer_str}' not found in label2id mapping.")
            # Option 2: Assign a default/unknown ID if you have one, or skip sample.
            # For now, let's make it return something that might cause a downstream error if not handled,
            # or you can choose to filter these out in collate_fn or raise error.
            # To proceed, we'll assume for now this case should be an error or filtered.
            # For robust training, you'd need a clear strategy.
            raise ValueError(f"Answer '{processed_answer_str}' (from original '{row['Answer']}') not found in label2id mapping for image {row['Image_Path']}.")


        return {
            "image": pil_image,          # Return the PIL Image object
            "question": question_text,   # Return the raw question string
            "labels": torch.tensor(answer_id, dtype=torch.long) # Return the label as a tensor
        }

In [7]:
""" Prepare dataloaders """
from functools import partial

# Use extended_label2id instead of reloading the model
num_labels = len(extended_label2id)

# Directory containing the images
image_dir = "/kaggle/input/filtered-small-amazon-qna"

# Create datasets with EXTENDED labels
train_dataset = QnADataset(train_df, image_dir, processor, extended_label2id)
val_dataset = QnADataset(val_df_filtered, image_dir, processor, extended_label2id)
test_dataset = QnADataset(test_df_filtered, image_dir, processor, extended_label2id)

# Corrected collate function with proper argument order
def collate_fn(batch, processor, num_classes=num_labels):
    """ViLT-compatible collate function with one-hot encoding"""
    # Filter out invalid entries
    valid_batch = [
        item for item in batch 
        if item is not None 
        and isinstance(item.get("image"), Image.Image)
        and item.get("question") 
        and item.get("labels") is not None
    ]
    
    if not valid_batch:
        return None
    
    # Process valid items
    images = [item["image"] for item in valid_batch]
    texts = [item["question"] for item in valid_batch]
    labels = [item["labels"] for item in valid_batch]  # Should be class indices

    # Process through processor
    try:
        encoding = processor(
            images=images,
            text=texts,
            return_tensors="pt",
            padding="longest",
            truncation=True,
            max_length=512
        )
    except Exception as e:
        print(f"Skipping batch: {str(e)}")
        return None

    # Convert labels to one-hot encoding
    batch_size = len(labels)
    one_hot_labels = torch.zeros(batch_size, num_classes)
    for i, label in enumerate(labels):
        one_hot_labels[i, label] = 1.0

    encoding["labels"] = one_hot_labels
    return encoding

# Create DataLoaders with proper partial binding
train_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    collate_fn=partial(collate_fn, processor=processor),  # Keyword argument binding
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=partial(collate_fn, processor=processor)  # Keyword argument binding
)

test_loader = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=partial(collate_fn, processor=processor)  # Keyword argument binding
)

# Fine tuning part

In [12]:
import torch
import torch.optim as optim
from tqdm import tqdm
import os
import numpy as np
from peft import LoraConfig, get_peft_model

# --- Configuration ---
NUM_EPOCHS = 20
LEARNING_RATE = 1e-4
OUTPUT_DIR = "/kaggle/working/vilt-lora-manual-best"
os.makedirs(OUTPUT_DIR, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Model Setup ---
original_model = ViltForQuestionAnswering.from_pretrained(
    "dandelin/vilt-b32-finetuned-vqa",
    num_labels=len(extended_label2id),
    id2label=extended_id2label,
    label2id=extended_label2id,
    ignore_mismatched_sizes=True
)

# --- PEFT Configuration ---
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"]
)
peft_model = get_peft_model(original_model, lora_config)
peft_model.print_trainable_parameters()
peft_model.to(device)

# --- Optimizer ---
optimizer = optim.AdamW(peft_model.parameters(), lr=LEARNING_RATE)

print(f"\n--- Starting Manual Training for {NUM_EPOCHS} epochs ---")
best_val_loss = float('inf')

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch + 1}/{NUM_EPOCHS}")

    # --- Training Phase ---
    peft_model.train()
    total_train_loss, train_batches = 0, 0
    
    progress_bar_train = tqdm(train_loader, desc="Training", leave=False)
    for batch in progress_bar_train:
        # Skip invalid/empty batches
        if batch is None:
            print("Skipping empty training batch")
            continue

        try:
            # Move batch to device
            batch_on_device = {k: v.to(device) for k, v in batch.items()}
            
            # Forward pass
            optimizer.zero_grad()
            outputs = peft_model(**batch_on_device)
            loss = outputs.loss

            # Backward pass
            loss.backward()
            optimizer.step()

            # Update metrics
            total_train_loss += loss.item()
            train_batches += 1
            progress_bar_train.set_postfix(batch_loss=loss.item())

        except Exception as e:
            print(f"\nError in training batch: {str(e)}")
            print("Skipping problematic training batch")
            continue

    # --- Validation Phase ---
    peft_model.eval()
    total_val_loss, val_batches = 0, 0
    
    progress_bar_val = tqdm(val_loader, desc="Validation", leave=False)
    with torch.no_grad():
        for batch in progress_bar_val:
            # Skip invalid/empty batches
            if batch is None:
                print("Skipping empty validation batch")
                continue

            try:
                # Move batch to device
                batch_on_device = {k: v.to(device) for k, v in batch.items()}
                
                # Forward pass
                outputs = peft_model(**batch_on_device)
                loss = outputs.loss

                # Update metrics
                total_val_loss += loss.item()
                val_batches += 1
                progress_bar_val.set_postfix(batch_loss=loss.item())

            except Exception as e:
                print(f"\nError in validation batch: {str(e)}")
                print("Skipping problematic validation batch")
                continue

    # Calculate epoch metrics
    avg_train_loss = total_train_loss / train_batches if train_batches > 0 else float('inf')
    avg_val_loss = total_val_loss / val_batches if val_batches > 0 else float('inf')
    
    print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        print(f"New best model! Saving to {OUTPUT_DIR}")
        peft_model.save_pretrained(OUTPUT_DIR)

print("\n--- Training Finished ---")
print(f"Best Validation Loss: {best_val_loss:.4f}")
print(f"Model saved to: {OUTPUT_DIR}")

Using device: cuda


Some weights of ViltForQuestionAnswering were not initialized from the model checkpoint at dandelin/vilt-b32-finetuned-vqa and are newly initialized because the shapes did not match:
- classifier.3.weight: found shape torch.Size([3129, 1536]) in the checkpoint and torch.Size([3464, 1536]) in the model instantiated
- classifier.3.bias: found shape torch.Size([3129]) in the checkpoint and torch.Size([3464]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 6,803,336 || all params: 124,906,768 || trainable%: 5.4467

--- Starting Manual Training for 20 epochs ---

Epoch 1/20


KeyboardInterrupt: 

# Metrics

In [6]:
!pip install bert-score
!git clone https://github.com/neulab/BARTScore.git
import sys
sys.path.append("./BARTScore")
# Now import
from bart_score import BARTScorer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing in

In [8]:
import sys
import time
import torch
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# 1. Add local BARTScore code into Python’s import path
sys.path.append("./BARTScore")

# 2. Semantic‐similarity imports
from bert_score import score as bert_score
from bart_score import BARTScorer

# 3. PEFT & model imports
from transformers import ViltForQuestionAnswering
from peft import PeftModel

# 4. Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 5. Reload base + LoRA‐finetuned model
#    (assumes you previously saved to OUTPUT_DIR)
OUTPUT_DIR = "/kaggle/input/lora_vilt_best/transformers/default/1/vilt-lora-manual-best"
base_model = ViltForQuestionAnswering.from_pretrained(
    "dandelin/vilt-b32-finetuned-vqa",
    num_labels=len(extended_label2id),
    id2label=extended_id2label,
    label2id=extended_label2id,
    ignore_mismatched_sizes=True
)
model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)
model.to(device)
model.eval()

# 6. Accumulators
all_pred_ids   = []
all_true_ids   = []
all_pred_texts = []
all_true_texts = []

# --- Start overall timer ---
t0_overall = time.time()

# 7. Inference + gather labels/texts with progress bar
t0_loop = time.time()
for batch in tqdm(test_loader, desc="Evaluating batches"):
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    logits  = outputs.logits

    # Predicted & true IDs
    pred_ids = logits.argmax(dim=-1)
    true_ids = batch["labels"].argmax(dim=-1)

    # Flatten for metrics
    pred_flat = pred_ids.view(-1).cpu().numpy()
    true_flat = true_ids.view(-1).cpu().numpy()
    all_pred_ids.extend(pred_flat)
    all_true_ids.extend(true_flat)

    # Convert to label strings
    all_pred_texts.extend([model.config.id2label[i] for i in pred_flat])
    all_true_texts.extend([model.config.id2label[i] for i in true_flat])
t1_loop = time.time()
print(f"\nInference & gathering took {t1_loop - t0_loop:.2f}s")

# 8. Classification metrics
t0_cls = time.time()
accuracy  = accuracy_score(all_true_ids, all_pred_ids)
precision = precision_score(all_true_ids, all_pred_ids, average="macro", zero_division=0)
recall    = recall_score(all_true_ids, all_pred_ids, average="macro", zero_division=0)
f1        = f1_score(all_true_ids, all_pred_ids, average="macro", zero_division=0)
t1_cls = time.time()

print(f"\nClassification metrics computed in {t1_cls - t0_cls:.2f}s")
print("=== Classification Metrics ===")
print(f"Accuracy      : {accuracy:.4f}")
print(f"Precision (M) : {precision:.4f}")
print(f"Recall    (M) : {recall:.4f}")
print(f"F1 Score  (M) : {f1:.4f}")

# 9. BERTScore (semantic similarity)
t0_bert = time.time()
bert_p, bert_r, bert_f1 = bert_score(
    all_pred_texts,
    all_true_texts,
    lang="en",
    model_type="bert-base-uncased",
    rescale_with_baseline=True
)
t1_bert = time.time()
print(f"\nBERTScore computed in {t1_bert - t0_bert:.2f}s")
print("=== BERTScore ===")
print(f"Precision : {bert_p.mean().item():.4f}")
print(f"Recall    : {bert_r.mean().item():.4f}")
print(f"F1        : {bert_f1.mean().item():.4f}")

# 10. BARTScore (semantic entailment)
t0_bart = time.time()
bart_scorer = BARTScorer(device=device.type, checkpoint="facebook/bart-large-cnn")
bart_scores = bart_scorer.score(
    all_pred_texts,
    all_true_texts,
    batch_size=8
)
t1_bart = time.time()
mean_bart = sum(bart_scores) / len(bart_scores)
print(f"\nBARTScore computed in {t1_bart - t0_bart:.2f}s")
print("=== BARTScore ===")
print(f"Mean score: {mean_bart:.4f}")

# --- End overall timer ---
t1_overall = time.time()
print(f"\nTotal evaluation time: {t1_overall - t0_overall:.2f}s")


Using device: cuda


Some weights of ViltForQuestionAnswering were not initialized from the model checkpoint at dandelin/vilt-b32-finetuned-vqa and are newly initialized because the shapes did not match:
- classifier.3.weight: found shape torch.Size([3129, 1536]) in the checkpoint and torch.Size([3464, 1536]) in the model instantiated
- classifier.3.bias: found shape torch.Size([3129]) in the checkpoint and torch.Size([3464]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating batches: 100%|██████████| 416/416 [01:27<00:00,  4.74it/s]


Inference & gathering took 87.82s

Classification metrics computed in 0.02s
=== Classification Metrics ===
Accuracy      : 0.6367
Precision (M) : 0.0604
Recall    (M) : 0.0810
F1 Score  (M) : 0.0643


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]


BARTScore computed in 17.67s
=== BARTScore ===
Mean score: -3.5814

Total evaluation time: 112.58s


In [13]:
import shutil
import os

folder_to_zip = "/kaggle/working/vilt-lora-manual-best"
output_zip_name = "/kaggle/working/vilt-lora-manual-best_model" # Name for the zip file (no .zip here)

try:
    shutil.make_archive(output_zip_name,  # The name of the file to create (e.g., /kaggle/working/my_model_archive)
                        'zip',             # The format (zip, tar, etc.)
                        root_dir=os.path.dirname(folder_to_zip), # The directory containing the folder to zip
                        base_dir=os.path.basename(folder_to_zip)) # The folder to zip

    print(f"Successfully created zip file: {output_zip_name}.zip")
    print(f"You can now find '{os.path.basename(output_zip_name)}.zip' in the Output section on the right sidebar (or under /kaggle/working/) and download it.")
except FileNotFoundError:
    print(f"Error: The folder {folder_to_zip} was not found. Please check the path.")
except Exception as e:
    print(f"An error occurred during zipping: {e}")

# Optional: Display a download link (might not always work perfectly in all browsers/setups)
from IPython.display import FileLink, display
if os.path.exists(output_zip_name + ".zip"):
    print("\nAttempting to display a download link (click to download):")
    display(FileLink(output_zip_name + ".zip"))
else:
    print(f"\nZip file {output_zip_name}.zip not found for creating a direct link. Please check the Output section manually.")

Successfully created zip file: /kaggle/working/vilt-lora-manual-best_model.zip
You can now find 'vilt-lora-manual-best_model.zip' in the Output section on the right sidebar (or under /kaggle/working/) and download it.

Attempting to display a download link (click to download):


/kaggle/working/vilt-lora-manual-best_model.zip

In [20]:
import zipfile
import os

# Define the output zip file name
zip_filename = '/kaggle/working/output.zip'

# List of files and directories to include
items_to_zip = [
    'vilt-lora-manual-best',
    'README.md',
    'adapter_config.json',
    'adapter_model.safetensors'
]

def zip_directory(path, ziph):
    # Walk through all files in the directory
    for root, dirs, files in os.walk(path):
        for file in files:
            file_path = os.path.join(root, file)
            # Add file to zip file, preserving directory structure
            ziph.write(file_path, os.path.relpath(file_path, os.path.dirname(path)))

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for item in items_to_zip:
        if os.path.isdir(item):
            # Add directory recursively
            zip_directory(item, zipf)
        elif os.path.isfile(item):
            # Add single file
            zipf.write(item)
            
print(f"Zip file created at: {zip_filename}")

Zip file created at: /kaggle/working/output.zip


In [19]:
!ls -lh /kaggle/working/

total 25M
drwxr-xr-x 2 root root 4.0K May 10 18:08 vilt-lora-manual-best
-rw-r--r-- 1 root root  25M May 11 03:58 vilt-lora-manual-best_model.zip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
exit()